In [26]:
import pandas as pd
import numpy as np
import requests
from google_api import get_lat_lon
import numpy as np
from string import digits

# Rental housing market in Canada

This project is related to exploring the price of housing market in Canada. 

Cleaning English Canada

In [15]:
import pandas as pd
import glob
path="C:/Users/mehdi/courses/"

In [16]:
from pathlib import Path
folder="C:/Users/mehdi/courses/rental project/English_canada"
li=[]
for file in Path(folder).glob('*.csv'):
    print(file)
    df = pd.read_csv(file)
    li.append(df)
df=pd.concat(li,axis=0,ignore_index=True).drop(columns=['Unnamed: 0'])



C:\Users\mehdi\courses\rental project\English_canada\Abbotsford.csv
C:\Users\mehdi\courses\rental project\English_canada\Adolphe.csv
C:\Users\mehdi\courses\rental project\English_canada\Brampton.csv
C:\Users\mehdi\courses\rental project\English_canada\Burnaby.csv
C:\Users\mehdi\courses\rental project\English_canada\Calgary.csv
C:\Users\mehdi\courses\rental project\English_canada\chilliwack.csv
C:\Users\mehdi\courses\rental project\English_canada\Edmonton.csv
C:\Users\mehdi\courses\rental project\English_canada\For_Mcmurray.csv
C:\Users\mehdi\courses\rental project\English_canada\Grande_prairie.csv
C:\Users\mehdi\courses\rental project\English_canada\Hamilton.csv
C:\Users\mehdi\courses\rental project\English_canada\lethbridge.csv
C:\Users\mehdi\courses\rental project\English_canada\London.csv
C:\Users\mehdi\courses\rental project\English_canada\mississauga.csv
C:\Users\mehdi\courses\rental project\English_canada\Moose_jaw.csv
C:\Users\mehdi\courses\rental project\English_canada\New_west

In [17]:
df

,price,Address,moreinfo,bedroom,bathroom,size_sqf,types,city,province
0,"1,425",Poplar,1 bd\n1 ba\n533 sf,1,1,533,Apt/Condo,Abbotsford,BC
1,"2,245",Poplar,2 bd\n2 ba\n910 sf,2,2,910,Apt/Condo,Abbotsford,BC
2,"1,350",Abbotsford West,1 bd\n1 ba\n538 sf,1,1,538,Apt/Condo,Abbotsford,BC
3,"1,725",Poplar,1 bd\n1 ba\n689 sf,1,1,689,Apt/Condo,Abbotsford,BC
4,"1,625",Central Abbotsford,2 bd\n1 ba\n846 sf,2,1,846,Apt/Condo,Abbotsford,BC
...,...,...,...,...,...,...,...,...,...
4987,600,Yorkton,0 bd\n1 ba\n0 sf,0,1,0,Apt/Condo,yorkton,SK
4988,761,Yorkton,1 bd\n1 ba\n0 sf,1,1,0,Apt/Condo,yorkton,SK
4989,951,Yorkton,3 bd\n1 ba\n806 sf,3,1,806,Apt/Condo,yorkton,SK
4990,951,Yorkton,3 bd\n1 ba\n806 sf,3,1,806,Apt/Condo,yorkton,SK


In [18]:
df=df.dropna()
df['city'].str.count('Toronto').sum()
df['moreinfo'] = df['moreinfo'].str.strip()
df['moreinfo'].str.count('V').sum()
df['moreinfo'] = df['moreinfo'].replace(" ","", regex=True)
df['moreinfo'] = df['moreinfo'].replace("\n","", regex=True)
df['moreinfo'] = df['moreinfo'].str.strip()
df['bathroom']=df['moreinfo'].apply(lambda r: r.split('bd')[1].split('ba')[0])
df=df.dropna()
df['price'] = df['price'].map(lambda x: x.replace(',',''))
df['price'] = df['price'].astype(int)
df['bedroom']=df['bedroom'].apply(lambda r:int(r) if isinstance(r,int) else r.split('+')[0])
df['bedroom']=df['bedroom'].astype(int)
#df['bathroom'] = df['bathroom'].replace(r'^([A-Za-z]|[0-9]|_)+$', np.NaN, regex=True)
#df['bathroom']=df['bathroom'].astype(int)
df['size_sqf'] = df['size_sqf'].replace('<', np.NaN, regex=True)
df=df.dropna()
df['size_sqf'] = df['size_sqf'].replace('5000+',5000,regex=True)
df['size_sqf'] = df['size_sqf'].astype(int)
df=df[df['size_sqf']>100]
for col in ['Address', 'types', 'city', 'province']:
    df[col] = df[col].astype('category')
df=df.reset_index()
df=df.drop(['moreinfo'], axis=1)

<ipython-input-18-51730f6f39bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['moreinfo'] = df['moreinfo'].str.strip()
<ipython-input-18-51730f6f39bd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['moreinfo'] = df['moreinfo'].replace(" ","", regex=True)
<ipython-input-18-51730f6f39bd>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [19]:
df

,index,price,Address,bedroom,bathroom,size_sqf,types,city,province
0,0,1425,Poplar,1,1,533,Apt/Condo,Abbotsford,BC
1,1,2245,Poplar,2,2,910,Apt/Condo,Abbotsford,BC
2,2,1350,Abbotsford West,1,1,538,Apt/Condo,Abbotsford,BC
3,3,1725,Poplar,1,1,689,Apt/Condo,Abbotsford,BC
4,4,1625,Central Abbotsford,2,1,846,Apt/Condo,Abbotsford,BC
...,...,...,...,...,...,...,...,...,...
2736,4984,801,Yorkton,1,1,502,Apt/Condo,yorkton,SK
2737,4986,926,Yorkton,2,1,709,Apt/Condo,yorkton,SK
2738,4989,951,Yorkton,3,1,806,Apt/Condo,yorkton,SK
2739,4990,951,Yorkton,3,1,806,Apt/Condo,yorkton,SK


In [20]:
df.dtypes

index          int64
price          int32
Address     category
bedroom        int32
bathroom      object
size_sqf       int32
types       category
city        category
province    category
dtype: object

Cleaning Montreal Data

In [21]:
folder="C:/Users/mehdi/courses/rental project/Montreal"
li=[]
for file in Path(folder).glob('*.csv'):
    print(file)
    df1 = pd.read_csv(file)
    li.append(df1)
df1=pd.concat(li,axis=0,ignore_index=True).drop(columns=['Unnamed: 0'])



C:\Users\mehdi\courses\rental project\Montreal\Montreal_apt.csv
C:\Users\mehdi\courses\rental project\Montreal\Montreal_house.csv


In [22]:
df1

,price,Address,bedroom,bathroom,size_sqf,types,city,province
0,2200,370 Rue des Seigneurs,1,1,804.0,Apt/Condo,Montreal,Quebec
1,2745,3660 Av. du Musee,2,1,900.0,Apt/Condo,Montreal,Quebec
2,2350,370 Rue des Seigneurs,1,1,888.0,Apt/Condo,Montreal,Quebec
3,3069,4300 Boul. De Maisonneuve O.,2,2,1292.0,Apt/Condo,Montreal,Quebec
4,2115,6850 Boul. Newman,3,2,1166.0,Apt/Condo,Montreal,Quebec
...,...,...,...,...,...,...,...,...
641,1535,6250 Av. Lennox,1,1,700.0,House,Montreal,Quebec
642,6999,400 Rue Sherbrooke O.,2,2,1175.0,House,Montreal,Quebec
643,1437,6325 Av. Somerled,2,1,770.0,House,Montreal,Quebec
644,3500,358 Tsse St-Denis,3,2,1800.0,House,Montreal,Quebec


In [27]:
df1['size_sqf'] = df1['size_sqf'].astype(int)
remove_digits = str.maketrans('', '', digits)
df1['Address']=df1['Address'].map(lambda x: x.translate(remove_digits)) ##removing bulding number
for col in ['Address', 'types', 'city', 'province']:
    df1[col] = df1[col].astype('category')

In [28]:
df1


,price,Address,bedroom,bathroom,size_sqf,types,city,province
0,2200,Rue des Seigneurs,1,1,804,Apt/Condo,Montreal,Quebec
1,2745,Av. du Musee,2,1,900,Apt/Condo,Montreal,Quebec
2,2350,Rue des Seigneurs,1,1,888,Apt/Condo,Montreal,Quebec
3,3069,Boul. De Maisonneuve O.,2,2,1292,Apt/Condo,Montreal,Quebec
4,2115,Boul. Newman,3,2,1166,Apt/Condo,Montreal,Quebec
...,...,...,...,...,...,...,...,...
641,1535,Av. Lennox,1,1,700,House,Montreal,Quebec
642,6999,Rue Sherbrooke O.,2,2,1175,House,Montreal,Quebec
643,1437,Av. Somerled,2,1,770,House,Montreal,Quebec
644,3500,Tsse St-Denis,3,2,1800,House,Montreal,Quebec


Merging two datasets

In [29]:
df=df.append(df1)
df=df.reset_index()
df=df.drop(['index'],axis=1)
df=df.drop(['level_0'],axis=1)

In [30]:
df

,price,Address,bedroom,bathroom,size_sqf,types,city,province
0,1425,Poplar,1,1,533,Apt/Condo,Abbotsford,BC
1,2245,Poplar,2,2,910,Apt/Condo,Abbotsford,BC
2,1350,Abbotsford West,1,1,538,Apt/Condo,Abbotsford,BC
3,1725,Poplar,1,1,689,Apt/Condo,Abbotsford,BC
4,1625,Central Abbotsford,2,1,846,Apt/Condo,Abbotsford,BC
...,...,...,...,...,...,...,...,...
3382,1535,Av. Lennox,1,1,700,House,Montreal,Quebec
3383,6999,Rue Sherbrooke O.,2,2,1175,House,Montreal,Quebec
3384,1437,Av. Somerled,2,1,770,House,Montreal,Quebec
3385,3500,Tsse St-Denis,3,2,1800,House,Montreal,Quebec


In [31]:
df.dtypes
df['bathroom']=df['bathroom'].astype(int)

In [32]:
df.describe ()

,price,bedroom,bathroom,size_sqf
count,3387.000000,3387.000000,3387.000000,3387.000000
mean,2808.636256,1.708592,1.383820,1064.153528
std,20698.354927,0.953958,0.732551,11168.660178
min,260.000000,0.000000,0.000000,103.000000
25%,1150.500000,1.000000,1.000000,604.000000
50%,1650.000000,2.000000,1.000000,780.000000
75%,2350.000000,2.000000,2.000000,980.000000
max,615000.000000,8.000000,11.000000,650230.000000


# adding lantitude and longitude

In [ ]:
df["Latitude"] = np.nan
df["Longitude"] = np.nan
for i in df.index:
    try:
        adres='{0},{1}, {2}, Canada'.format(df['Address'][i],df['city'][i],df['province'][i])
        b=[]
        IP_key=''
        b=get_lat_lon(adres,IP_key)
        df['Latitude'][i]=b[0]
        df["Longitude"][i]=b[1]
    except:
        pass

In [ ]:
df.to_csv('cleaned.csv') #exporting the final data

'asd  as '